# second part of spark activity 

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

read the data from the (ml-100k/u.data) file : 

In [5]:
textFile = sc.textFile("../data/raw/Datasets/ml-100k/u.data")
#data Split
rdd = textFile.map(lambda line: line.split("\t"))
# explore user rating  
print("User ratings (user id, item id, rating, timestamp)"+format(rdd.first()))

User ratings (user id, item id, rating, timestamp)['196', '242', '3', '881250949']


now i will convert the rating data into float and integer, and explore them again:

In [7]:
converted_rdd = rdd.map(lambda fields: (int(float(fields[0])), float(fields[2]), int(float(fields[1]))))
# Print user rating after convert
print("Numerical user ratings (user id, rating, item id):" + format(converted_rdd.first())) 

Numerical user ratings (user id, rating, item id):(196, 3.0, 242)


create a pair RDD:

In [8]:
paired_rdd = converted_rdd.map(lambda fields: (fields[0], (fields[1], fields[2])))

print("Pair RDD (user id, (rating, item id))" + format(paired_rdd.first()))

Pair RDD (user id, (rating, item id))(196, (3.0, 242))



sum of rating, and the number of rated movies for each user


In [9]:
aggregated_rdd = paired_rdd.aggregateByKey((0.0, 0.0),\
                               lambda acc, val: (val[0] + acc[0], acc[1] + 1),\
                               lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))

print("Agregate user ratings and movies ((user id, (rating sum, number of movies)):" 
      + format(aggregated_rdd.takeOrdered(5, lambda x: x[0])))

Agregate user ratings and movies ((user id, (rating sum, number of movies)):[(1, (982.0, 272.0)), (2, (230.0, 62.0)), (3, (151.0, 54.0)), (4, (104.0, 24.0)), (5, (503.0, 175.0))]



average of ratings by movie for each user


In [10]:
avg_rdd = aggregated_rdd.mapValues(lambda x:(x[0]/x[1]))

print("average ratings of user((user id, rating sum/number of movies):" 
      + format(avg_rdd.takeOrdered(5, lambda x: x[0])))


average ratings of user((user id, rating sum/number of movies):[(1, 3.610294117647059), (2, 3.7096774193548385), (3, 2.7962962962962963), (4, 4.333333333333333), (5, 2.874285714285714)]


the received rating of each movie

In [11]:
movies_rating = paired_rdd.map(lambda x : (x[1][1], 1)).reduceByKey(lambda a,b: a+b)

print("Movie number of ratings (movie id, ratings number):" 
      + format(movies_rating.take(5)))

Movie number of ratings (movie id, ratings number):[(242, 117), (302, 297), (377, 13), (51, 81), (346, 126)]


number of movies which have a higher than average rating

In [12]:
high_rating_movies = paired_rdd.map(lambda x : (x[1][1], x[1][0])).filter(lambda x : x[1] >= 4.0)\
           .map(lambda x : (x[0], 1)).reduceByKey(lambda a,b: a+b)

print("High Rating Movies:" 
      + format(high_rating_movies.takeOrdered(5, lambda x: x[0])))

High Rating Movies:[(1, 321), (2, 51), (3, 34), (4, 122), (5, 39)]


Top 5 the Last 5 rated movies

In [14]:
print("Top 5 Rated movies:" 
      + format(movies_rating.takeOrdered(5, lambda x: -x[1])))

Top 5 Rated movies:[(50, 583), (258, 509), (100, 508), (181, 507), (294, 485)]


In [15]:
print("Last 5 Rated movies:" 
      + format(movies_rating.takeOrdered(5, lambda x: x[1])))

Last 5 Rated movies:[(1348, 1), (1320, 1), (1492, 1), (1364, 1), (1493, 1)]


Join the two movie_counts and high_rating_movies datasets using a leftOuterJoin

In [16]:
def generate_ratio(x, y):
    try:
       return float(x)/float(y)
    except:
        return 0 #x or y are None

In [18]:
join_movies = movies_rating.leftOuterJoin(high_rating_movies)

print("Join the two movie_counts and high_rating_movies datasets using a leftOuterJoin:" 
     + format(join_movies.take(5)))

print("Movie with id = 314:" 
     + format(join_movies.filter(lambda x : x[0] == 314).collect()))

print("Top higher rates movie (movie id, (high ratings number/ ratings number):" 
     + format(join_movies.mapValues(lambda x : (generate_ratio(x[1], x[0])))\
           .takeOrdered(10, lambda x: -x[1])))

Join the two movie_counts and high_rating_movies datasets using a leftOuterJoin:[(242, (117, 89)), (302, (297, 239)), (377, (13, 1)), (51, (81, 45)), (346, (126, 77))]
Movie with id = 314:[(314, (5, None))]
Top higher rates movie (movie id, (high ratings number/ ratings number):[(1492, 1.0), (1189, 1.0), (1498, 1.0), (1500, 1.0), (814, 1.0), (711, 1.0), (1449, 1.0), (1125, 1.0), (1536, 1.0), (1396, 1.0)]
